In [10]:
import os, sys 
sys.path.append('..')

import random 
import robust as rb 
import torch as ch 
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from nff.io.ase_calcs import NeuralFF, AtomsBatch, EnsembleNFF
from nff.data import Dataset
from nff.train import load_model 

from ase.io import Trajectory, read 


In [11]:
# converting xyz to dataset 
def _xyz_to_dataset(xyz_file):
    # read the xyz file
    atoms_list = read(xyz_file, index=':')

    # initialize the props dictionary 
    props = {
        'nxyz' : [],
        'energy' : [],
        'energy_grad' : [],
        'stress' : [],
        'lattice' : [],
    }
    
    for atoms in atoms_list:
        # get the atomic number and positions
        numbers = atoms.get_atomic_numbers()
        positions = atoms.get_positions()
        
        #get the lattice
        lattice = atoms.get_cell().array
        props['lattice'].append(lattice)

        # combine the atomic numbers and positions 
        nxyz = np.column_stack((numbers, positions))
        props['nxyz'].append(nxyz)

        # xtract the energy
        energy = atoms.info['REF_energy']
        props['energy'].append(energy)
        
        # get the energy gradient
        forces = atoms.todict()['REF_force']
        energy_grad = -forces  # Note the negative sign
        props['energy_grad'].append(energy_grad)

        # Extract stress if available
        if atoms.info['REF_stress'].any():
            stress = atoms.info['REF_stress']
            props['stress'].append(stress)
        else:
            props['stress'].append(None)
        
        # create the dataset object
        dataset = Dataset(props=props, units='eV')
        
        return dataset
    

import numpy as np
import torch
from ase.io import read
from nff.data import Dataset


def xyz_to_dataset(xyz_file):
    # Read the XYZ file
    atoms_list = read(xyz_file, index=':')
    
    # Initialize the props dictionary
    props = {
        'nxyz': [],
        'energy': [],
        'energy_grad': [],
        'stress': [],
        'lattice': [],
        'num_atoms': []  # Add this to keep track of the number of atoms in each structure
    }
    
    for atoms in atoms_list:
        # Extract atomic numbers and positions
        numbers = atoms.get_atomic_numbers()
        positions = atoms.get_positions()
        
        # Get the lattice
        lattice = atoms.get_cell().array
        props['lattice'].append(torch.tensor(lattice, dtype=torch.float32))
        
        # Combine atomic numbers and positions into nxyz format
        nxyz = np.column_stack((numbers, positions))
        props['nxyz'].append(torch.tensor(nxyz, dtype=torch.float32))
        
        # Extract energy
        energy = atoms.info['REF_energy']
        props['energy'].append(torch.tensor(energy, dtype=torch.float32))
        
        # Extract forces and convert to energy gradient
        forces = atoms.todict()['REF_force']
        energy_grad = -forces  # Note the negative sign
        props['energy_grad'].append(torch.tensor(energy_grad, dtype=torch.float32))
        
        # Extract stress if available
        if atoms.info['REF_stress'].any():
            stress = atoms.info['REF_stress']
            props['stress'].append(torch.tensor(stress, dtype=torch.float32))
        else:
            props['stress'].append(torch.tensor([0.0, 0.0, 0.0, 0.0, 0.0, 0.0], dtype=torch.float32))
        
        # Add the number of atoms in this structure
        props['num_atoms'].append(torch.tensor(len(atoms), dtype=torch.int64))
    
    # Convert single-value properties to tensors
    props['energy'] = torch.tensor(props['energy'])
    props['stress'] = torch.stack(props['stress'])
    props['lattice'] = torch.stack(props['lattice'])
    props['num_atoms'] = torch.tensor(props['num_atoms'])
    
    # Keep multi-dimensional properties as lists of tensors
    # props['nxyz'] and props['energy_grad'] are already lists of tensors
    
    # Create the Dataset object
    dataset = Dataset(props=props, units='eV')
    
    return dataset

In [12]:
models = []
for i in range(8):
    model_path = f'finished_runs/2024-09-27_vcrtiwzr_fep+vac+neb+perf/model_{i}/vcrtiwzr_all_e1_f50_25s_seed_{i}_compiled.model'
    m = NeuralFF.from_file(model_path,device='cpu',).model
    models.append(m)

In [13]:
ensemble = EnsembleNFF(models,device='cpu')

In [14]:
dset = xyz_to_dataset('data/fep_vac_neb_perf_test.xyz')

In [15]:
CUTOFF = 6

def get_atoms(props):
    atoms = AtomsBatch(
        positions=props['nxyz'][:,1:],
        numbers=props['nxyz'][:,0],
        cell=props['lattice'],
        pbc=True,
        cutoff=CUTOFF,
        props={'energy': 0, 'energy_grad': [], 'stress': []},
        calculator=ensemble,
        device='cpu',
    )
    _ = atoms.update_nbr_list()

    return atoms 

initial = get_atoms(random.choice(dset))

In [16]:
# define the attacker
energies_per_atom = ch.tensor(dset.props['energy']) / dset.props['num_atoms']
#forces = ch.tensor(dset.props['energy_grad'])

energy_dset = rb.PotentialDataset(
    ch.zeros_like(energies_per_atom),
    energies_per_atom,
    energies_per_atom,
)

loss_fn = rb.loss.AdvLoss(
    train=energy_dset,
    temperature=20,
)


In [17]:
attacker = rb.schnet.Attacker(
    initial,
    ensemble,
    loss_fn,
    device='cpu',
)

In [18]:
results = attacker.attack(epochs=10)

  0%|          | 0/10 [00:00<?, ?it/s]


RuntimeError: forward() Expected a value of type 'Dict[str, Tensor]' for argument 'data' but instead found type 'dict'.
Position: 1
Value: {'energy': tensor(0.), 'energy_grad': tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [